In [9]:
import pandas as pd

# Step 1: Load the data
cleaned_customer_df = pd.read_csv("cleaned_customers.csv")
cleaned_stores_df = pd.read_csv("cleaned_stores.csv")
cleaned_sales_df = pd.read_csv("cleaned_sales.csv")
cleaned_products_df = pd.read_csv("cleaned_products.csv")
cleaned_exchange_rates_df = pd.read_csv("cleaned_Exchange_Rates.csv")

# Step 2: Rename 'State' and 'Country' in cleaned_stores_df to avoid conflicts
cleaned_stores_df.rename(columns={'State': 'Store_State', 'Country': 'Store_Country'}, inplace=True)

# Step 3: Merge sales with customer data on 'CustomerKey'
merged_df = cleaned_sales_df.merge(cleaned_customer_df, on='CustomerKey', how='left')

# Step 4: Merge the result with store data on 'StoreKey'
merged_df = merged_df.merge(cleaned_stores_df, on='StoreKey', how='left')

# Step 5: Merge with product data on 'ProductKey'
merged_df = merged_df.merge(cleaned_products_df, on='ProductKey', how='left')

# Step 8: Proceed with the merge on 'Order Date' and 'Currency Code'
merged_df = merged_df.merge(
    cleaned_exchange_rates_df, 
    left_on=['Order Date', 'Currency Code'], 
    right_on=['Date', 'Currency'], 
    how='left'
)

# Step 9: Drop the extra 'Date' and 'Currency' columns from exchange rates
merged_df = merged_df.drop(columns=['Date', 'Currency'])

merged_df['Actual_Unitcost_In_USD'] = (merged_df['Unit Cost USD'] * merged_df['Exchange']).round(2)
merged_df['Actual_Unitprice_In_USD'] = (merged_df['Unit Price USD'] * merged_df['Exchange']).round(2)

# Step 10: Output the first few rows of the final merged DataFrame
print(merged_df.head())


   Order Number  Line Item  Order Date  Delivery Date  CustomerKey  StoreKey  \
0        366000          1  2016-01-01  Not Delivered       265598        10   
1        366001          1  2016-01-01      1/13/2016      1269051         0   
2        366001          2  2016-01-01      1/13/2016      1269051         0   
3        366002          1  2016-01-01      1/12/2016       266019         0   
4        366002          2  2016-01-01      1/12/2016       266019         0   

   ProductKey  Quantity Currency Code  Gender  ...   Color Unit Cost USD  \
0        1304         1           CAD    Male  ...   White         31.27   
1        1048         2           USD    Male  ...  Silver        141.47   
2        2007         1           USD    Male  ...   Black        220.64   
3        1106         7           CAD  Female  ...  Orange        148.08   
4         373         1           CAD  Female  ...   White        166.20   

  Unit Price USD SubcategoryKey                       Subcateg

In [10]:
# Step 1: Collect columns from all original DataFrames
original_columns = (
    set(cleaned_customer_df.columns) |
    set(cleaned_stores_df.columns) |
    set(cleaned_sales_df.columns) |
    set(cleaned_products_df.columns) |
    set(cleaned_exchange_rates_df.columns)
)

# Step 2: Collect columns from the merged DataFrame
merged_columns = set(merged_df.columns)

# Step 3: Find missing columns (present in original but not in merged)
missing_columns = original_columns - merged_columns

# Step 4: Find extra columns (present in merged but not in original)
extra_columns = merged_columns - original_columns

# Step 5: Display results
print(f"Missing columns in merged DataFrame: {missing_columns}")
print(f"Extra columns in merged DataFrame: {extra_columns}")


Missing columns in merged DataFrame: {'Date', 'Currency'}
Extra columns in merged DataFrame: {'Actual_Unitcost_In_USD', 'Actual_Unitprice_In_USD'}


In [11]:
# Save the merged DataFrame to a CSV file
merged_df.to_csv("merged_data.csv", index=False)

print("Merged DataFrame has been saved to 'merged_data.csv'.")


Merged DataFrame has been saved to 'merged_data.csv'.
